# 使用三种API开放平台实现人脸识别

* 使用的单人示例图片
![image](https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=3165450438,2790296485&fm=11&gp=0.jpg)
* 使用的多人示例图片
![image](https://dss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=2907717037,1924937543&fm=111&gp=0.jpg)

## Azure

- 单人人脸识别数据

In [1]:
import requests
import json

# set to your own subscription key value
subscription_key = '03cc7cc2313342d4831cbad7ca6ac426'
assert subscription_key

# replace <My Endpoint String> with the string from your endpoint URL
face_api_url = 'https://eastus.api.cognitive.microsoft.com/face/v1.0/detect'

image_url = 'https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=3165450438,2790296485&fm=11&gp=0.jpg'

headers = {'Ocp-Apim-Subscription-Key': subscription_key}

params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}

response = requests.post(face_api_url, params=params,
                         headers=headers, json={"url": image_url})
print(json.dumps(response.json()))

[{"faceId": "7c2896c0-d984-40a1-81f6-13c8841d3f86", "faceRectangle": {"top": 105, "left": 181, "width": 68, "height": 68}, "faceAttributes": {"smile": 0.0, "headPose": {"pitch": 4.2, "roll": 5.2, "yaw": -14.8}, "gender": "male", "age": 23.0, "facialHair": {"moustache": 0.1, "beard": 0.1, "sideburns": 0.1}, "glasses": "NoGlasses", "emotion": {"anger": 0.0, "contempt": 0.0, "disgust": 0.0, "fear": 0.0, "happiness": 0.0, "neutral": 1.0, "sadness": 0.0, "surprise": 0.0}, "blur": {"blurLevel": "low", "value": 0.18}, "exposure": {"exposureLevel": "goodExposure", "value": 0.58}, "noise": {"noiseLevel": "low", "value": 0.0}, "makeup": {"eyeMakeup": true, "lipMakeup": true}, "accessories": [], "occlusion": {"foreheadOccluded": false, "eyeOccluded": false, "mouthOccluded": false}, "hair": {"bald": 0.13, "invisible": false, "hairColor": [{"color": "black", "confidence": 1.0}, {"color": "gray", "confidence": 0.66}, {"color": "other", "confidence": 0.63}, {"color": "brown", "confidence": 0.33}, {"c

- 多人人脸识别数据

In [2]:
import requests
import json

# set to your own subscription key value
subscription_key = '03cc7cc2313342d4831cbad7ca6ac426'
assert subscription_key

# replace <My Endpoint String> with the string from your endpoint URL
face_api_url = 'https://eastus.api.cognitive.microsoft.com/face/v1.0/detect'

image_url = 'https://dss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=2907717037,1924937543&fm=111&gp=0.jpg'

headers = {'Ocp-Apim-Subscription-Key': subscription_key}

params2 = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}
response2 = requests.post(face_api_url, params=params2,
                         headers=headers, json={"url": image_url})
print(json.dumps(response2.json()))

[{"faceId": "772034e3-0b9c-463f-866b-b076f38009ea", "faceRectangle": {"top": 87, "left": 102, "width": 147, "height": 147}, "faceAttributes": {"smile": 0.0, "headPose": {"pitch": -6.0, "roll": 3.7, "yaw": 31.6}, "gender": "male", "age": 23.0, "facialHair": {"moustache": 0.1, "beard": 0.1, "sideburns": 0.1}, "glasses": "NoGlasses", "emotion": {"anger": 0.011, "contempt": 0.003, "disgust": 0.001, "fear": 0.0, "happiness": 0.0, "neutral": 0.922, "sadness": 0.062, "surprise": 0.0}, "blur": {"blurLevel": "low", "value": 0.0}, "exposure": {"exposureLevel": "goodExposure", "value": 0.61}, "noise": {"noiseLevel": "low", "value": 0.1}, "makeup": {"eyeMakeup": true, "lipMakeup": true}, "accessories": [], "occlusion": {"foreheadOccluded": false, "eyeOccluded": false, "mouthOccluded": false}, "hair": {"bald": 0.07, "invisible": false, "hairColor": [{"color": "black", "confidence": 0.99}, {"color": "other", "confidence": 0.93}, {"color": "gray", "confidence": 0.46}, {"color": "red", "confidence": 0

## 腾讯云

- 单人人脸识别数据

In [29]:
import hashlib  
import time  
import random  
import string
import requests  
import base64  
import requests

import numpy as np
from urllib.parse import urlencode
import json #用于post后得到的字符串到字典的转换

app_id = '2131690713' 
app_key = 'YDtKcITUF0dF49d3'

'''
        腾讯openai鉴权签名计算步骤：（摘抄自官网）
            用于计算签名的参数在不同接口之间会有差异，但算法过程固定如下4个步骤。
        1 将<key, value>请求参数对按key进行字典升序排序，得到有序的参数对列表N
        2 将列表N中的参数对按URL键值对的格式拼接成字符串，得到字符串T（如：key1=value1&key2=value2），URL键值拼接过程value部分需要URL编码，URL编码算法用大写字母，例如%E8，而不是小写%e8
        3 将应用密钥以app_key为键名，组成URL键值拼接到字符串T末尾，得到字符串S（如：key1=value1&key2=value2&app_key=密钥)
        4 对字符串S进行MD5运算，将得到的MD5值所有字符转换成大写，得到接口请求签名
'''

def face_check(img_data):
    """
    人脸识别demo
    :param img_data: 二进制的图片数据
    :return:
    """

    data = base64.b64encode(img_data)

    image = data.decode()

    imageType = "BASE64"
    
def get_params(img):                         #鉴权计算并返回请求参数
    #请求时间戳（秒级），用于防止请求重放（保证签名5分钟有效
    time_stamp=str(int(time.time())) 
    #请求随机字符串，用于保证签名不可预测,16代表16位
    nonce_str = ''.join(random.sample(string.ascii_letters + string.digits, 16))

    params3 = {'app_id':app_id,                #请求包，需要根据不同的任务修改，基本相同
              'image':img,                    #文字类的任务可能是‘text’，由主函数传递进来
              'mode':'0' ,                    #身份证件类可能是'card_type'
              'time_stamp':time_stamp,        #时间戳，都一样
              'nonce_str':nonce_str,          #随机字符串，都一样
              #'sign':''                      #签名不参与鉴权计算，只是列出来示意
             }

    sort_dict= sorted(params3.items(), key=lambda item:item[0], reverse = False)  #字典排序
    sort_dict.append(('app_key',app_key))   #尾部添加appkey
    rawtext= urlencode(sort_dict).encode()  #urlencod编码
    sha = hashlib.md5()    
    sha.update(rawtext)
    md5text= sha.hexdigest().upper()        #MD5加密计算
    params3['sign']=md5text                  #将签名赋值到sign
    return  params3                          #返回请求包

with open(r"C:\Users\hp\Desktop\666.jpg","rb")as f:
    base64_data = base64.b64encode(f.read())
    
params3 = get_params(base64_data)

url = "https://api.ai.qq.com/fcgi-bin/face/face_detectface"
res = requests.post(url,params3).json()
print(res)

{'ret': 0, 'msg': 'ok', 'data': {'image_width': 440, 'image_height': 330, 'face_list': [{'face_id': '3591528478116158228', 'x': 178, 'y': 96, 'width': 84, 'height': 84, 'gender': 99, 'age': 20, 'expression': 0, 'beauty': 81, 'glass': 0, 'pitch': -6, 'yaw': -3, 'roll': -6, 'face_shape': {'face_profile': [{'x': 190, 'y': 121}, {'x': 188, 'y': 128}, {'x': 187, 'y': 134}, {'x': 187, 'y': 141}, {'x': 188, 'y': 148}, {'x': 189, 'y': 155}, {'x': 191, 'y': 161}, {'x': 194, 'y': 167}, {'x': 199, 'y': 173}, {'x': 204, 'y': 177}, {'x': 210, 'y': 179}, {'x': 218, 'y': 179}, {'x': 225, 'y': 177}, {'x': 232, 'y': 174}, {'x': 238, 'y': 169}, {'x': 242, 'y': 164}, {'x': 246, 'y': 157}, {'x': 248, 'y': 150}, {'x': 250, 'y': 142}, {'x': 251, 'y': 135}, {'x': 251, 'y': 128}], 'left_eye': [{'x': 196, 'y': 122}, {'x': 198, 'y': 124}, {'x': 201, 'y': 124}, {'x': 204, 'y': 124}, {'x': 207, 'y': 123}, {'x': 205, 'y': 120}, {'x': 201, 'y': 119}, {'x': 198, 'y': 120}], 'right_eye': [{'x': 238, 'y': 126}, {'x': 

- 多人人脸识别数据

In [4]:
import hashlib  
import time  
import random  
import string
import requests  
import base64  
import requests

import numpy as np
from urllib.parse import urlencode
import json #用于post后得到的字符串到字典的转换

app_id = '2131690713' 
app_key = 'YDtKcITUF0dF49d3'

'''
        腾讯openai鉴权签名计算步骤：（摘抄自官网）
            用于计算签名的参数在不同接口之间会有差异，但算法过程固定如下4个步骤。
        1 将<key, value>请求参数对按key进行字典升序排序，得到有序的参数对列表N
        2 将列表N中的参数对按URL键值对的格式拼接成字符串，得到字符串T（如：key1=value1&key2=value2），URL键值拼接过程value部分需要URL编码，URL编码算法用大写字母，例如%E8，而不是小写%e8
        3 将应用密钥以app_key为键名，组成URL键值拼接到字符串T末尾，得到字符串S（如：key1=value1&key2=value2&app_key=密钥)
        4 对字符串S进行MD5运算，将得到的MD5值所有字符转换成大写，得到接口请求签名
'''

def face_check(img_data):
    """
    人脸识别demo
    :param img_data: 二进制的图片数据
    :return:
    """

    data = base64.b64encode(img_data)

    image = data.decode()

    imageType = "BASE64"
    
def get_params(img):                         #鉴权计算并返回请求参数
    #请求时间戳（秒级），用于防止请求重放（保证签名5分钟有效
    time_stamp=str(int(time.time())) 
    #请求随机字符串，用于保证签名不可预测,16代表16位
    nonce_str = ''.join(random.sample(string.ascii_letters + string.digits, 16))

    params4 = {'app_id':app_id,                #请求包，需要根据不同的任务修改，基本相同
              'image':img,                    #文字类的任务可能是‘text’，由主函数传递进来
              'mode':'0' ,                    #身份证件类可能是'card_type'
              'time_stamp':time_stamp,        #时间戳，都一样
              'nonce_str':nonce_str,          #随机字符串，都一样
              #'sign':''                      #签名不参与鉴权计算，只是列出来示意
             }

    sort_dict= sorted(params4.items(), key=lambda item:item[0], reverse = False)  #字典排序
    sort_dict.append(('app_key',app_key))   #尾部添加appkey
    rawtext= urlencode(sort_dict).encode()  #urlencod编码
    sha = hashlib.md5()    
    sha.update(rawtext)
    md5text= sha.hexdigest().upper()        #MD5加密计算
    params4['sign']=md5text                  #将签名赋值到sign
    return  params4                         #返回请求包

with open(r"C:\Users\hp\Desktop\777.jpg","rb")as f:
    base64_data = base64.b64encode(f.read())
    
params4 = get_params(base64_data)

url = "https://api.ai.qq.com/fcgi-bin/face/face_detectface"
res2 = requests.post(url,params4).json()
print(res2)

{'ret': 0, 'msg': 'ok', 'data': {'image_width': 500, 'image_height': 334, 'face_list': [{'face_id': '3591505377727905191', 'x': 59, 'y': 66, 'width': 181, 'height': 181, 'gender': 99, 'age': 23, 'expression': 36, 'beauty': 84, 'glass': 0, 'pitch': 10, 'yaw': 24, 'roll': 11, 'face_shape': {'face_profile': [{'x': 72, 'y': 143}, {'x': 76, 'y': 160}, {'x': 82, 'y': 176}, {'x': 89, 'y': 192}, {'x': 98, 'y': 207}, {'x': 110, 'y': 218}, {'x': 125, 'y': 227}, {'x': 141, 'y': 233}, {'x': 158, 'y': 237}, {'x': 175, 'y': 237}, {'x': 191, 'y': 232}, {'x': 199, 'y': 222}, {'x': 201, 'y': 210}, {'x': 203, 'y': 198}, {'x': 206, 'y': 186}, {'x': 209, 'y': 175}, {'x': 212, 'y': 163}, {'x': 213, 'y': 151}, {'x': 213, 'y': 139}, {'x': 211, 'y': 126}, {'x': 208, 'y': 116}], 'left_eye': [{'x': 125, 'y': 131}, {'x': 132, 'y': 133}, {'x': 139, 'y': 133}, {'x': 146, 'y': 132}, {'x': 153, 'y': 130}, {'x': 147, 'y': 125}, {'x': 139, 'y': 124}, {'x': 131, 'y': 126}], 'right_eye': [{'x': 202, 'y': 117}, {'x': 199

## Face++ 

- 单人人脸识别数据

In [5]:
import requests


api_secret = "zj8IdhGXyDBhovPklDI7IujNMCIaXbj1"
api_key = '9cibumhegbiNlgMu6EREIYvoDCdjmYpV'  # Replace with a valid Subscription Key here.

BASE_URL = 'https://api-cn.faceplusplus.com/facepp/v3/detect' 
img_url = 'https://ss3.bdstatic.com/70cFv8Sh_Q1YnxGkpoWK1HF6hhy/it/u=3165450438,2790296485&fm=11&gp=0.jpg'

headers = {
    'Content-Type': 'application/json',
}

payload = {
    "image_url":img_url,
    'api_key': api_key,
    'api_secret': api_secret,
    'return_attributes':'gender,age,smiling,emotion', 
}

r = requests.post(BASE_URL, params=payload, headers=headers)

r.status_code
print(r.content)

b'{"request_id":"1586678413,8303687f-b39a-4d8c-a1ed-e3eb6cb8e2c7","time_used":1615,"faces":[{"face_token":"73235503620b0906a51a9c601889e823","face_rectangle":{"top":111,"left":184,"width":69,"height":69},"attributes":{"gender":{"value":"Male"},"age":{"value":23},"smile":{"value":0.000,"threshold":50.000},"emotion":{"anger":0.043,"disgust":0.110,"fear":7.423,"happiness":0.196,"neutral":91.465,"sadness":0.482,"surprise":0.281}}}],"image_id":"R+L3q2mPZQ2hLptWYsoaxA==","face_num":1}\n'


- 多人人脸识别数据

In [6]:
import requests


api_secret = "zj8IdhGXyDBhovPklDI7IujNMCIaXbj1"
api_key = '9cibumhegbiNlgMu6EREIYvoDCdjmYpV'  # Replace with a valid Subscription Key here.

BASE_URL = 'https://api-cn.faceplusplus.com/facepp/v3/detect' 
img_url = 'https://dss2.bdstatic.com/70cFvnSh_Q1YnxGkpoWK1HF6hhy/it/u=2907717037,1924937543&fm=111&gp=0.jpg'

headers = {
    'Content-Type': 'application/json',
}

payload2 = {
    "image_url":img_url,
    'api_key': api_key,
    'api_secret': api_secret,
    'return_attributes':'gender,age,smiling,emotion', 
}

r2 = requests.post(BASE_URL, params=payload2, headers=headers)

r2.status_code
print(r2.content)

b'{"request_id":"1586678418,10150b7e-5494-4f10-b2a0-78a9d84be282","time_used":115,"faces":[{"face_token":"8f481823755fc14d994c22a7fa75c66e","face_rectangle":{"top":103,"left":83,"width":139,"height":139},"attributes":{"gender":{"value":"Male"},"age":{"value":20},"smile":{"value":3.945,"threshold":50.000},"emotion":{"anger":0.001,"disgust":0.002,"fear":0.001,"happiness":0.003,"neutral":99.993,"sadness":0.001,"surprise":0.000}}},{"face_token":"94bd64d2e6943cd29cc76e864987f8fc","face_rectangle":{"top":121,"left":297,"width":136,"height":136},"attributes":{"gender":{"value":"Male"},"age":{"value":23},"smile":{"value":0.191,"threshold":50.000},"emotion":{"anger":0.009,"disgust":0.075,"fear":0.009,"happiness":0.071,"neutral":99.708,"sadness":0.059,"surprise":0.069}}}],"image_id":"pF2Grcvj0FZOEj/WxhQv4g==","face_num":2}\n'


# 运用Azure分析人脸数据制作数据表格

- 单人人脸识别分析

In [7]:
results = response.json()
results

[{'faceId': '7c2896c0-d984-40a1-81f6-13c8841d3f86',
  'faceRectangle': {'top': 105, 'left': 181, 'width': 68, 'height': 68},
  'faceAttributes': {'smile': 0.0,
   'headPose': {'pitch': 4.2, 'roll': 5.2, 'yaw': -14.8},
   'gender': 'male',
   'age': 23.0,
   'facialHair': {'moustache': 0.1, 'beard': 0.1, 'sideburns': 0.1},
   'glasses': 'NoGlasses',
   'emotion': {'anger': 0.0,
    'contempt': 0.0,
    'disgust': 0.0,
    'fear': 0.0,
    'happiness': 0.0,
    'neutral': 1.0,
    'sadness': 0.0,
    'surprise': 0.0},
   'blur': {'blurLevel': 'low', 'value': 0.18},
   'exposure': {'exposureLevel': 'goodExposure', 'value': 0.58},
   'noise': {'noiseLevel': 'low', 'value': 0.0},
   'makeup': {'eyeMakeup': True, 'lipMakeup': True},
   'accessories': [],
   'occlusion': {'foreheadOccluded': False,
    'eyeOccluded': False,
    'mouthOccluded': False},
   'hair': {'bald': 0.13,
    'invisible': False,
    'hairColor': [{'color': 'black', 'confidence': 1.0},
     {'color': 'gray', 'confidence'

In [8]:
import pandas as pd
df = pd.json_normalize(results)
df = df.rename (columns = {"faceAttributes.smile":"笑容","faceAttributes.headPose.roll":"头部偏向（上下）",
                           "faceAttributes.facialHair.beard":"胡须",
                           "faceAttributes.facialHair.sideburns":"胡须角",
                           "faceAttributes.glasses":"佩戴眼镜",
                           "faceAttributes.emotion.anger":"愤怒",
                           "faceAttributes.emotion.contempt":"鄙视",
                           "faceAttributes.headPose.yaw":"头部偏向（左右）",
                           "faceAttributes.gender":"性别","faceAttributes.age":"年龄",
                           "faceAttributes.facialHair.moustache":"有无胡子",
                           "faceAttributes.noise.value":"噪音值",
                           "faceAttributes.makeup.eyeMakeup":"眼妆",
                           "faceAttributes.makeup.lipMakeup":"唇彩",
                           "faceAttributes.accessories":"配饰",
                           "faceAttributes.occlusion.foreheadOccluded":"前额闭塞",
                           "faceAttributes.occlusion.eyeOccluded":"眼睛开合",
                           "faceAttributes.occlusion.mouthOccluded":"嘴巴开合",
                           "faceAttributes.hair.bald":"秃头程度",
                           "faceAttributes.hair.invisible":"看不见头发",
                           "faceAttributes.hair.hairColor":"发色"})
df = df.set_index('faceId')
df = df.iloc[:,6:]
df.replace({"male":"男",
            "female":"女",
            "NoGlasses":"不佩戴眼镜",
            "Glasses":"佩戴眼镜",})

,头部偏向（上下）,头部偏向（左右）,性别,年龄,有无胡子,胡须,胡须角,佩戴眼镜,愤怒,鄙视,...,噪音值,眼妆,唇彩,配饰,前额闭塞,眼睛开合,嘴巴开合,秃头程度,看不见头发,发色
faceId,,,,,,,,,,,,,,,,,,,,,
7c2896c0-d984-40a1-81f6-13c8841d3f86,5.2,-14.8,男,23.0,0.1,0.1,0.1,不佩戴眼镜,0.0,0.0,...,0.0,True,True,[],False,False,False,0.13,False,"[{'color': 'black', 'confidence': 1.0}, {'colo..."


- 多人人脸识别分析

In [9]:
results2 = response2.json()
results2

[{'faceId': '772034e3-0b9c-463f-866b-b076f38009ea',
  'faceRectangle': {'top': 87, 'left': 102, 'width': 147, 'height': 147},
  'faceAttributes': {'smile': 0.0,
   'headPose': {'pitch': -6.0, 'roll': 3.7, 'yaw': 31.6},
   'gender': 'male',
   'age': 23.0,
   'facialHair': {'moustache': 0.1, 'beard': 0.1, 'sideburns': 0.1},
   'glasses': 'NoGlasses',
   'emotion': {'anger': 0.011,
    'contempt': 0.003,
    'disgust': 0.001,
    'fear': 0.0,
    'happiness': 0.0,
    'neutral': 0.922,
    'sadness': 0.062,
    'surprise': 0.0},
   'blur': {'blurLevel': 'low', 'value': 0.0},
   'exposure': {'exposureLevel': 'goodExposure', 'value': 0.61},
   'noise': {'noiseLevel': 'low', 'value': 0.1},
   'makeup': {'eyeMakeup': True, 'lipMakeup': True},
   'accessories': [],
   'occlusion': {'foreheadOccluded': False,
    'eyeOccluded': False,
    'mouthOccluded': False},
   'hair': {'bald': 0.07,
    'invisible': False,
    'hairColor': [{'color': 'black', 'confidence': 0.99},
     {'color': 'other', 

In [10]:
import pandas as pd
df2 = pd.json_normalize(results2)
df2 = df2.rename (columns = {"faceAttributes.smile":"笑容","faceAttributes.headPose.roll":"头部偏向（上下）",
                           "faceAttributes.facialHair.beard":"胡须",
                           "faceAttributes.facialHair.sideburns":"胡须角",
                           "faceAttributes.glasses":"佩戴眼镜",
                           "faceAttributes.emotion.anger":"愤怒",
                           "faceAttributes.emotion.contempt":"鄙视",
                           "faceAttributes.headPose.yaw":"头部偏向（左右）",
                           "faceAttributes.gender":"性别","faceAttributes.age":"年龄",
                           "faceAttributes.facialHair.moustache":"有无胡子",
                           "faceAttributes.noise.value":"噪音值",
                           "faceAttributes.makeup.eyeMakeup":"眼妆",
                           "faceAttributes.makeup.lipMakeup":"唇彩",
                           "faceAttributes.accessories":"配饰",
                           "faceAttributes.occlusion.foreheadOccluded":"前额闭塞",
                           "faceAttributes.occlusion.eyeOccluded":"眼睛开合",
                           "faceAttributes.occlusion.mouthOccluded":"嘴巴开合",
                           "faceAttributes.hair.bald":"秃头程度",
                           "faceAttributes.hair.invisible":"看不见头发",
                           "faceAttributes.hair.hairColor":"发色"})
df2 = df2.set_index('faceId')
df2 = df2.iloc[:,6:]
df2.replace({"male":"男",
            "female":"女",
            "NoGlasses":"不佩戴眼镜",
            "Glasses":"佩戴眼镜",})

,头部偏向（上下）,头部偏向（左右）,性别,年龄,有无胡子,胡须,胡须角,佩戴眼镜,愤怒,鄙视,...,噪音值,眼妆,唇彩,配饰,前额闭塞,眼睛开合,嘴巴开合,秃头程度,看不见头发,发色
faceId,,,,,,,,,,,,,,,,,,,,,
772034e3-0b9c-463f-866b-b076f38009ea,3.7,31.6,男,23.0,0.1,0.1,0.1,不佩戴眼镜,0.011,0.003,...,0.1,True,True,[],False,False,False,0.07,False,"[{'color': 'black', 'confidence': 0.99}, {'col..."
26d2f8cc-95f7-43fc-b9a7-ee2f56a4f2e4,-2.7,-1.3,男,24.0,0.1,0.1,0.1,不佩戴眼镜,0.000,0.036,...,0.0,True,True,[],False,False,False,0.11,False,"[{'color': 'black', 'confidence': 0.99}, {'col..."


# 运用腾讯云分析人脸数据制作数据表格

- 单人人脸识别

In [11]:
results5 = res
results5

{'ret': 0,
 'msg': 'ok',
 'data': {'image_width': 440,
  'image_height': 330,
  'face_list': [{'face_id': '3591505338030914982',
    'x': 178,
    'y': 96,
    'width': 84,
    'height': 84,
    'gender': 99,
    'age': 20,
    'expression': 0,
    'beauty': 81,
    'glass': 0,
    'pitch': -6,
    'yaw': -3,
    'roll': -6,
    'face_shape': {'face_profile': [{'x': 190, 'y': 121},
      {'x': 188, 'y': 128},
      {'x': 187, 'y': 134},
      {'x': 187, 'y': 141},
      {'x': 188, 'y': 148},
      {'x': 189, 'y': 155},
      {'x': 191, 'y': 161},
      {'x': 194, 'y': 167},
      {'x': 199, 'y': 173},
      {'x': 204, 'y': 177},
      {'x': 210, 'y': 179},
      {'x': 218, 'y': 179},
      {'x': 225, 'y': 177},
      {'x': 232, 'y': 174},
      {'x': 238, 'y': 169},
      {'x': 242, 'y': 164},
      {'x': 246, 'y': 157},
      {'x': 248, 'y': 150},
      {'x': 250, 'y': 142},
      {'x': 251, 'y': 135},
      {'x': 251, 'y': 128}],
     'left_eye': [{'x': 196, 'y': 122},
      {'x': 19

In [12]:
import pandas as pd
df5 = pd.json_normalize(results5)
df5 = df5.rename (columns = {"data.face_list":"列表"})
df5.replace({})

,ret,msg,data.image_width,data.image_height,列表
0,0,ok,440,330,"[{'face_id': '3591505338030914982', 'x': 178, ..."


- 多人人脸识别

In [13]:
results6 = res2
results6

{'ret': 0,
 'msg': 'ok',
 'data': {'image_width': 500,
  'image_height': 334,
  'face_list': [{'face_id': '3591505377727905191',
    'x': 59,
    'y': 66,
    'width': 181,
    'height': 181,
    'gender': 99,
    'age': 23,
    'expression': 36,
    'beauty': 84,
    'glass': 0,
    'pitch': 10,
    'yaw': 24,
    'roll': 11,
    'face_shape': {'face_profile': [{'x': 72, 'y': 143},
      {'x': 76, 'y': 160},
      {'x': 82, 'y': 176},
      {'x': 89, 'y': 192},
      {'x': 98, 'y': 207},
      {'x': 110, 'y': 218},
      {'x': 125, 'y': 227},
      {'x': 141, 'y': 233},
      {'x': 158, 'y': 237},
      {'x': 175, 'y': 237},
      {'x': 191, 'y': 232},
      {'x': 199, 'y': 222},
      {'x': 201, 'y': 210},
      {'x': 203, 'y': 198},
      {'x': 206, 'y': 186},
      {'x': 209, 'y': 175},
      {'x': 212, 'y': 163},
      {'x': 213, 'y': 151},
      {'x': 213, 'y': 139},
      {'x': 211, 'y': 126},
      {'x': 208, 'y': 116}],
     'left_eye': [{'x': 125, 'y': 131},
      {'x': 132, 

In [14]:
import pandas as pd
df6 = pd.json_normalize(results6)
df6 = df6.rename (columns = {"data.face_list":"列表"})
df6

,ret,msg,data.image_width,data.image_height,列表
0,0,ok,500,334,"[{'face_id': '3591505377727905191', 'x': 59, '..."


# 运用Face++分析人脸数据制作数据表格

- 单人人脸识别分析

In [15]:
results3 = r.json()
results3

{'request_id': '1586678413,8303687f-b39a-4d8c-a1ed-e3eb6cb8e2c7',
 'time_used': 1615,
 'faces': [{'face_token': '73235503620b0906a51a9c601889e823',
   'face_rectangle': {'top': 111, 'left': 184, 'width': 69, 'height': 69},
   'attributes': {'gender': {'value': 'Male'},
    'age': {'value': 23},
    'smile': {'value': 0.0, 'threshold': 50.0},
    'emotion': {'anger': 0.043,
     'disgust': 0.11,
     'fear': 7.423,
     'happiness': 0.196,
     'neutral': 91.465,
     'sadness': 0.482,
     'surprise': 0.281}}}],
 'image_id': 'R+L3q2mPZQ2hLptWYsoaxA==',
 'face_num': 1}

In [16]:
import pandas as pd
df3 = pd.json_normalize(results3,record_path='faces')
df3 = df3.rename (columns = {"attributes.gender.value":"性别",
                           "attributes.age.value":"年龄",
                           "attributes.smile.value":"笑容",
                           "attributes.smile.threshold":"是否有笑容",
                           "attributes.emotion.anger":"愤怒",
                           "attributes.emotion.disgust":"厌恶",
                           "attributes.emotion.fear":"恐惧",
                           "attributes.emotion.happiness":"高兴",
                           "attributes.emotion.neutral":"平静",
                           "attributes.emotion.sadness":"悲伤",
                           "attributes.emotion.surprise":"惊讶"})
df3 = df3.set_index('face_token')
df3 = df3.iloc[:,4:]
df3.replace({"Male":"男",
            "Female":"女",})

,性别,年龄,笑容,是否有笑容,愤怒,厌恶,恐惧,高兴,平静,悲伤,惊讶
face_token,,,,,,,,,,,
73235503620b0906a51a9c601889e823,男,23,0.0,50.0,0.043,0.11,7.423,0.196,91.465,0.482,0.281


- 多人人脸识别

In [17]:
results4 = r2.json()
results4

{'request_id': '1586678418,10150b7e-5494-4f10-b2a0-78a9d84be282',
 'time_used': 115,
 'faces': [{'face_token': '8f481823755fc14d994c22a7fa75c66e',
   'face_rectangle': {'top': 103, 'left': 83, 'width': 139, 'height': 139},
   'attributes': {'gender': {'value': 'Male'},
    'age': {'value': 20},
    'smile': {'value': 3.945, 'threshold': 50.0},
    'emotion': {'anger': 0.001,
     'disgust': 0.002,
     'fear': 0.001,
     'happiness': 0.003,
     'neutral': 99.993,
     'sadness': 0.001,
     'surprise': 0.0}}},
  {'face_token': '94bd64d2e6943cd29cc76e864987f8fc',
   'face_rectangle': {'top': 121, 'left': 297, 'width': 136, 'height': 136},
   'attributes': {'gender': {'value': 'Male'},
    'age': {'value': 23},
    'smile': {'value': 0.191, 'threshold': 50.0},
    'emotion': {'anger': 0.009,
     'disgust': 0.075,
     'fear': 0.009,
     'happiness': 0.071,
     'neutral': 99.708,
     'sadness': 0.059,
     'surprise': 0.069}}}],
 'image_id': 'pF2Grcvj0FZOEj/WxhQv4g==',
 'face_num': 

In [18]:
import pandas as pd
df4 = pd.json_normalize(results4,record_path='faces')
df4 = df4.rename (columns = {"attributes.gender.value":"性别",
                           "attributes.age.value":"年龄",
                           "attributes.smile.value":"笑容",
                           "attributes.smile.threshold":"是否有笑容",
                           "attributes.emotion.anger":"愤怒",
                           "attributes.emotion.disgust":"厌恶",
                           "attributes.emotion.fear":"恐惧",
                           "attributes.emotion.happiness":"高兴",
                           "attributes.emotion.neutral":"平静",
                           "attributes.emotion.sadness":"悲伤",
                           "attributes.emotion.surprise":"惊讶"})
df4 = df4.set_index('face_token')
df4 = df4.iloc[:,4:]
df4.replace({"Male":"男",
            "Female":"女",})

,性别,年龄,笑容,是否有笑容,愤怒,厌恶,恐惧,高兴,平静,悲伤,惊讶
face_token,,,,,,,,,,,
8f481823755fc14d994c22a7fa75c66e,男,20,3.945,50.0,0.001,0.002,0.001,0.003,99.993,0.001,0.000
94bd64d2e6943cd29cc76e864987f8fc,男,23,0.191,50.0,0.009,0.075,0.009,0.071,99.708,0.059,0.069


---
# 3家API不同之处的结论

* 通过上面三家人脸识别api的数据对比我们可以看到：
    - Azure：可以分析图像中人物头部方向、人物脸部妆容、发量等特殊数据。
    - 腾讯云：定位人像五官的位置，给出精确定位数据。
    - Face++：分析多种情感，可以同时识别不同人脸情感状态。